In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [ ]:
tf.executing_eagerly()

In [ ]:
import pandas as pd
import cv2
import os.path
from PIL import Image
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
import os, logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  
from os import listdir
import glob
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
#import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
a = tf.zeros([], tf.float32)

## Imports libs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from IPython.display import clear_output
%load_ext tensorboard

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))


In [ ]:
!pwd

In [ ]:
path1 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

# renameImageFiles(path1, prefix1,'png')
# renameImageFiles(path2, prefix2,'png')
# renameImageFiles(path2, prefix2,'jpg')

In [ ]:
path3 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/Shap/CT_COVID/'
prefix3 = 'ct_covid'
path4 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/Shap/CT_NonCOVID/'
prefix4 = 'ct_noncovid'

In [ ]:
def processImages(imgDirPath, binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_features_df =  processImages(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImages(path2,0)#0 ---> covnid-19 negative

In [ ]:
ct_covid_features_df_shap =  processImages(path3,1)#1--> covid-19 positive
ct_noncovid_features_df_shap =  processImages(path4,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset)

In [ ]:
cvd_imgs_shap = [ct_noncovid_features_df_shap, ct_covid_features_df_shap]
cvd_imgs_dataset_shap = pd.concat(cvd_imgs_shap)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap = cvd_imgs_dataset_shap.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap)

#### Prepare negative covid images for machine learning ready

In [ ]:
input_data_x_ = ct_noncovid_features_df.iloc[:,:-1].to_numpy().reshape((2173,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_noncovid_features_df['output_encode'] = label_encoder.fit_transform(ct_noncovid_features_df['class_label'])
ct_noncovid_features_df
ct_noncovid_features_df = pd.get_dummies(ct_noncovid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_ = np.array(ct_noncovid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_negative_only = input_data_x_
output_label_y_negative_only = output_label_y_
print('Input_x Data Shape: \n{0}'.format(input_data_x_negative_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_negative_only.shape))

#### Prepare positive covid images for machine learning ready

In [ ]:
input_data_x__ = ct_covid_features_df.iloc[:,:-1].to_numpy().reshape((2476,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_covid_features_df['output_encode'] = label_encoder.fit_transform(ct_covid_features_df['class_label'])
ct_covid_features_df
ct_covid_features_df = pd.get_dummies(ct_covid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y__ = np.array(ct_covid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_positive_only = input_data_x__
output_label_y_positive_only = output_label_y__
print('Input_x Data Shape: \n{0}'.format(input_data_x_positive_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_positive_only.shape))

#### Prepare both positive and negetive covid images for machine learning ready

In [ ]:
input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((4649,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset['class_label'])
cvd_imgs_dataset
cvd_imgs_dataset = pd.get_dummies(cvd_imgs_dataset, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y = np.array(cvd_imgs_dataset[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y.shape))

In [ ]:
cvd_imgs_dataset

#### Prepare both positive and negetive covid shap images for machine learning ready

In [ ]:
input_data_x_shap = cvd_imgs_dataset_shap.iloc[:,:-1].to_numpy().reshape((3718,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap['class_label'])
cvd_imgs_dataset_shap
cvd_imgs_dataset_shap = pd.get_dummies(cvd_imgs_dataset_shap, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_shap = np.array(cvd_imgs_dataset_shap[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x_shap.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_shap.shape))

In [ ]:
cvd_imgs_dataset_shap

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(
    input_data_x, output_label_y, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_shap, test_features_shap, train_labels_shap, test_labels_shap = train_test_split(
    input_data_x_shap, output_label_y_shap, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_negative_only, test_features_negative_only, train_labels_negative_only, test_labels_negative_only = train_test_split(
    input_data_x_negative_only, output_label_y_negative_only, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_positive_only, test_features_positive_only, train_labels_positive_only, test_labels__positive_only = train_test_split(
    input_data_x_positive_only, output_label_y_positive_only, test_size=.20, random_state=42)

In [ ]:
train_features_shap.shape

In [ ]:
train_labels

In [ ]:
print(type(train_features_positive_only[0]))

### Loading the dataset

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from torchvision import transforms

class CustomImageDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        """
        Args:
            features (array-like): Input image features (e.g., images as numpy arrays).
            labels (array-like): Corresponding labels for the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        image = self.features[idx]
        label = self.labels[idx]

        # Ensure the image has the correct shape
        if image.ndim == 2:  # Grayscale image (height, width)
            image = np.expand_dims(image, axis=-1)  # Convert to (height, width, 1)
        elif image.ndim == 3 and image.shape[2] == 1:  # If the image is (height, width, 1)
            image = np.squeeze(image, axis=-1)  # Convert to (height, width)

        # Convert to PIL image
        if isinstance(image, np.ndarray):
            if image.ndim == 2:  # Grayscale
                image = Image.fromarray(image)  # Convert from numpy array to PIL Image
            elif image.ndim == 3:  # RGB
                image = Image.fromarray(image.astype('uint8'), 'RGB')

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
from torchvision import transforms

# Custom normalization for grayscale images (mean and std for single channel)
# normalize_grayscale = transforms.Normalize(mean=[0.485], std=[0.229])

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     normalize_grayscale,  # Apply normalization only for grayscale images
# ])

# Use a conditional normalization based on the number of channels
def dynamic_normalize(tensor):
    """
    Normalize tensors dynamically based on the number of channels.
    Args:
        tensor: A PyTorch tensor representing the image.
    Returns:
        Normalized tensor.
    """
    if tensor.shape[0] == 3:  # RGB Image
        return transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(tensor)
    elif tensor.shape[0] == 1:  # Grayscale Image
        return transforms.Normalize(mean=[0.485], std=[0.229])(tensor)
    else:
        raise ValueError(f"Unsupported number of channels: {tensor.shape[0]}")

# Add this dynamic normalization into your transform chain:
transform = transforms.Compose([
    transforms.ToTensor(),
    dynamic_normalize,  # Apply normalization dynamically based on channels
])

In [ ]:
# train_dataset = CustomImageDataset(features=train_features, labels=train_labels, transform=transform)
# #test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# # Create DataLoader instances
# loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# #test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


### Load datset for NonCOVID

In [ ]:
train_dataset_negative = CustomImageDataset(features=train_features_negative_only, labels=train_labels_negative_only, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_negative = DataLoader(train_dataset_negative, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


### Load datset for COVID

In [ ]:
train_dataset_positive = CustomImageDataset(features=train_features_positive_only, labels=train_labels_positive_only, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_positive = DataLoader(train_dataset_positive, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
diffusion_model = DiffusionModel(input_size=300*300).to(device)

In [ ]:
diffusion_model

### Building the multimodal

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# U-Net based Diffusion Model (Generator)
class DiffusionGenerator(nn.Module):
    def __init__(self, input_channels=3, output_channels=3):
        super(DiffusionGenerator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, output_channels, kernel_size=2, stride=2),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# GAN Generator
class GANGenerator(nn.Module):
    def __init__(self, latent_dim, img_size):
        super(GANGenerator, self).__init__()
        self.init_size = img_size // 4
        self.fc = nn.Linear(latent_dim, 128 * self.init_size ** 2)
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.fc(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

# GAN Discriminator
class Discriminator(nn.Module):
    def __init__(self, img_size):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(256 * (img_size // 8) * (img_size // 8), 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity

# Training Loop for the Multimodal Model
def train_multimodal_model(diffusion_generator, gan_generator, discriminator, dataloader, gen_optimizer, disc_optimizer, num_epochs, latent_dim):
    adversarial_loss = nn.BCELoss()
    for epoch in range(num_epochs):
        for real_images, _ in tqdm(dataloader):
            batch_size = real_images.size(0)
            valid = torch.ones(batch_size, 1, device=real_images.device)
            fake = torch.zeros(batch_size, 1, device=real_images.device)
            
            # Train Discriminator
            disc_optimizer.zero_grad()
            real_images = real_images.to(real_images.device)
            real_loss = adversarial_loss(discriminator(real_images), valid)

            # Generate fake images using GAN Generator
            z = torch.randn(batch_size, latent_dim, device=real_images.device)
            fake_images_gan = gan_generator(z)
            fake_loss = adversarial_loss(discriminator(fake_images_gan.detach()), fake)
            disc_loss = (real_loss + fake_loss) / 2
            disc_loss.backward()
            disc_optimizer.step()

            # Train GAN Generator
            gen_optimizer.zero_grad()
            gen_loss = adversarial_loss(discriminator(fake_images_gan), valid)
            gen_loss.backward()
            gen_optimizer.step()

            # Train Diffusion Generator (Optional step for separate diffusion training)
            diffusion_optimizer = optim.Adam(diffusion_generator.parameters(), lr=1e-4)
            diffusion_optimizer.zero_grad()
            fake_images_diffusion = diffusion_generator(real_images)
            diffusion_loss = nn.MSELoss()(fake_images_diffusion, real_images)
            diffusion_loss.backward()
            diffusion_optimizer.step()

# Initialize models and optimizers
img_size = 64
latent_dim = 100
num_epochs = 50

diffusion_generator = DiffusionGenerator().to("cuda")
gan_generator = GANGenerator(latent_dim, img_size).to("cuda")
discriminator = Discriminator(img_size).to("cuda")

gen_optimizer = optim.Adam(gan_generator.parameters(), lr=1e-4)
disc_optimizer = optim.Adam(discriminator.parameters(), lr=1e-4)



# Train the model
train_multimodal_model(diffusion_generator, gan_generator, discriminator, dataloader, gen_optimizer, disc_optimizer, num_epochs, latent_dim)


In [ ]:
!pwd